In [1]:
import yfinance as yf
import datetime
import time

aapl = yf.Ticker('AAPL')

In [16]:
aapl.info

{'zip': '95014',
 'sector': 'Technology',
 'fullTimeEmployees': 147000,
 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand r

In [20]:
aapl.info['sector']

'Technology'

In [9]:
# get current date and date 60 days ago
current_date = datetime.datetime.now()
current_date = current_date.strftime("%Y-%m-%d")
sixty_days_ago = (datetime.datetime.now() - datetime.timedelta(days=59))
sixty_days_ago = sixty_days_ago.strftime('%Y-%m-%d')

aapl.history(interval='15m', auto_adjust=True, start=sixty_days_ago, end=current_date)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2021-02-08 09:30:00-05:00,136.029999,136.250000,134.940094,135.736206,10408110,0,0
2021-02-08 09:45:00-05:00,135.625000,135.800003,135.360001,135.690002,4316546,0,0
2021-02-08 10:00:00-05:00,135.679993,136.309998,135.600006,135.958405,3860834,0,0
2021-02-08 10:15:00-05:00,135.960007,136.220001,135.720993,135.910004,2529242,0,0
2021-02-08 10:30:00-05:00,135.909393,136.630005,135.880005,136.590805,2967968,0,0
...,...,...,...,...,...,...,...
2021-04-05 14:45:00-04:00,125.848503,125.910004,125.599998,125.629997,1875123,0,0
2021-04-05 15:00:00-04:00,125.635002,125.720001,125.464996,125.628403,2353948,0,0
2021-04-05 15:15:00-04:00,125.625000,125.820000,125.480003,125.549004,2312180,0,0


In [2]:
# import SQLite

import sqlite3
from sqlite3 import Error

conn = sqlite3.connect('stockPrediction.db')

In [13]:
# build SQLite tables

# build table if not exists

# table for sectors, industries
conn.execute('''CREATE TABLE IF NOT EXISTS sector (
    sector_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    sector TEXT NOT NULL
);
''')

conn.execute('''CREATE TABLE IF NOT EXISTS industry (
    industry_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    industry TEXT NOT NULL
);
''')

# stock table
conn.execute('''CREATE TABLE IF NOT EXISTS stock (
    stock_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    stock_symbol TEXT NOT NULL,
    short_name TEXT,
    full_name TEXT,
    sector_id TEXT,
    industry_id TEXT,
    datetime_added TEXT NOT NULL,
    FOREIGN KEY (sector_id)
        REFERENCES sector (sector_id)
        ON UPDATE CASCADE
        ON DELETE CASCADE,
    FOREIGN KEY (industry_ID)
        REFERENCES industry (industry_id)
        ON UPDATE CASCADE
        ON DELETE CASCADE
);''')

conn.execute('''CREATE TABLE IF NOT EXISTS price_history (
    price_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    stock_id INTEGER NOT NULL,
    price_datetime TEXT NOT NULL,
    open_price REAL NOT NULL,
    high_price REAL NOT NULL,
    low_price REAL NOT NULL,
    close_price REAL NOT NULL,
    volume INTEGER NOT NULL,
    dividends REAL,
    stock_splits TEXT,
    datetime_added TEXT NOT NULL,
    FOREIGN KEY (stock_id)
        REFERENCES stock (stock_id)
        ON UPDATE CASCADE
        ON DELETE CASCADE
);
''')



## Add the stocks from a text file to the database, and get info regarding their sectors, industries, etc.

In [27]:
from stock import Stock
from stock import Price
from stock import get_current_datetime
import sqlite3
from sqlite3 import Error
import yfinance as yf
import datetime
import time

conn = sqlite3.connect('stockPrediction.db')

# open a list of symbols, each symbol as a single line
symbols = open('allUnderFiveDollarsAndPrevOwned - Copy.txt')

dbq = conn.execute('SELECT stock_symbol FROM stock')
dbq = dbq.fetchall()

db_symbols = [x[0] for x in dbq]

# loop thru the symbols
for symbol in symbols:

    symbol = symbol.strip('\n')

    if symbol in db_symbols:
        continue

    # call to yf to get the data
    data = yf.Ticker(symbol)

    # take the data and create a new Stock object w/ it, just to keep organized and help myself w/ OOP
    # try/except block for symbols that have since been delisted
    try:
        symbol = Stock(symbol, short_name=data.info['shortName'], full_name=data.info['longName'], sector=data.info['sector'], industry=data.info['industry'])
    except KeyError:
        print(f'{symbol} no longer a listed security')
        continue

    # trying this to see if I can correct my db locked error:
    conn.commit()

    # check to see if this security's industry/sector are already in the DB, if not, add them
    # must have a DB connection open prior to running loop
    symbol.check_industry(symbol.industry, conn)
    symbol.check_sector(symbol.sector, conn)

    # remove rogue single quotes from the short/full names
    symbol.short_name = symbol.short_name.replace("'", "")
    symbol.full_name = symbol.full_name.replace("'", "")

    # add the symbol to the db
    conn.execute(f"INSERT INTO stock (stock_symbol, short_name, full_name, sector_id, industry_id, datetime_added) VALUES ('{symbol.symbol}', '{symbol.short_name}', '{symbol.full_name}', {symbol.sector_id}, {symbol.industry_id}, '{get_current_datetime()}')")
    conn.commit()
    print(f'Wrote {symbol.symbol} to the DB')

    # pause to not get us blocked by yf
    time.sleep(3)
print('Done.')

the DB
Wrote MMJFF to the DB
Wrote MMLP to the DB
Wrote MMMB to the DB
Wrote MMND to the DB
Wrote MMNFF to the DB
Wrote MMRGF to the DB
Wrote MMSDF to the DB
Wrote MMTIF to the DB
Wrote MMTV to the DB
Wrote MNDJF to the DB
Wrote MNDO to the DB
Wrote MNKD to the DB
Wrote MNRLF to the DB
Wrote MNTR to the DB
Wrote MNZO to the DB
Wrote MODC to the DB
Wrote MOGO to the DB
Wrote MOGU to the DB
Wrote MOHO to the DB
Wrote MOST to the DB
Wrote MOSY to the DB
Wrote MOTS to the DB
Wrote MOXC to the DB
Wrote MPIR to the DB
Wrote MPXOF to the DB
Wrote MRAM to the DB
Wrote MREO to the DB
Wrote MRIN to the DB
Wrote MRKR to the DB
Wrote MRMD to the DB
MRQRF no longer a listed security
Wrote MRZM to the DB
Wrote MSLM to the DB
Wrote MSN to the DB
Wrote MSTF to the DB
Wrote MTC to the DB
Wrote MTENY to the DB
Wrote MTL to the DB
Wrote MTL/P to the DB
Wrote MTLFF to the DB
Wrote MTNB to the DB
Wrote MTP to the DB
Wrote MTR to the DB
Wrote MTRO to the DB
Wrote MTSL to the DB
Wrote MUFG to the DB
Wrote MU

In [24]:
symbol.short_name

SyntaxError: invalid syntax (<ipython-input-24-97b9a5578b4e>, line 1)

In [12]:
# this was to test to see if i wrote my list comprehension correctly

from stock import Stock
from stock import Price
from stock import get_current_datetime
import sqlite3
from sqlite3 import Error
import yfinance as yf
import datetime
import time

conn = sqlite3.connect('stockPrediction.db')

# open a list of symbols, each symbol as a single line
symbols = open('allUnderFiveDollarsAndPrevOwned - Copy.txt')

dbq = conn.execute('SELECT stock_symbol FROM stock')
dbq = dbq.fetchall()

db_symbols = [x[0] for x in dbq]

In [8]:
get_current_datetime()

'2021-04-10 13:24:56"'

In [2]:
# this is me testing my stock class
from stock import Stock
from stock import Price

aapl = Stock('AAPL')


In [3]:
print(aapl.symbol)

AAPL


In [4]:
aapl.short_name = 'Apple Inc.'

In [5]:
print(aapl.short_name)

Apple Inc.


In [10]:
symbol.symbol

'BLPH'

In [18]:
symbol.sector_id

(1,)

In [1]:
# testing class methods for stock
from stock import Stock
import yfinance as yf
import sqlite3

conn = sqlite3.connect('stockPrediction.db')

data = yf.Ticker('AAPL')

aapl = Stock(data.info['symbol'])
aapl.industry = data.info['industry']
aapl.sector = data.info['sector']

In [8]:
aapl.industry

'Consumer Electronics'

In [9]:
aapl.sector

'Technology'

In [2]:
aapl.check_industry(aapl.industry, conn)

In [3]:
print(aapl.industry_id)

(10,)


In [3]:
import yfinance as yf
import datetime

current_date = datetime.datetime.now().strftime("%Y-%m-%d")
sixty_days_ago = (datetime.datetime.now() - datetime.timedelta(days=59)).strftime("%Y-%m-%d")

data = yf.Ticker("BDSI")

data.history(
    start = sixty_days_ago,
    end = current_date,
    interval = '15m',
    auto_adjust = True
)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2021-02-11 09:30:00-05:00,4.2500,4.2599,4.1900,4.2250,91238,0,0
2021-02-11 09:45:00-05:00,4.2200,4.2250,4.1500,4.2075,33797,0,0
2021-02-11 10:00:00-05:00,4.2050,4.2097,4.1600,4.1700,30697,0,0
2021-02-11 10:15:00-05:00,4.1700,4.2150,4.1700,4.2020,19221,0,0
2021-02-11 10:30:00-05:00,4.2023,4.2100,4.2000,4.2100,12370,0,0
...,...,...,...,...,...,...,...
2021-04-09 14:45:00-04:00,3.5700,3.5700,3.5400,3.5635,55558,0,0
2021-04-09 15:00:00-04:00,3.5600,3.5700,3.5503,3.5550,28426,0,0
2021-04-09 15:15:00-04:00,3.5500,3.5600,3.5500,3.5500,54386,0,0
